<img src='https://www.riiid.co/assets/opengraph.png' width='700'>

<h1><center><strong>A Deeper Dive Into Riiid!: EDA|Models|Blending </strong></center><h1>
    
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:30px;background:black; border:0; color:white' role="tab" aria-controls="home"><center>Problem Statement</center></h2>
    
<div align='left'><font size="3" color="#000000">In this competition, your challenge is to create algorithms for "Knowledge Tracing," the modeling of student knowledge over time. The goal is to accurately predict how students will perform on future interactions. You will pair your machine learning skills using Riiid’s EdNet data.</font></div>
<hr>    
<div align='left'><font size="4" color="#000000"><h1 style="text-transform: uppercase"><strong> AIM: </strong> </font></div></h1>
<div align='left'><font size="3" color="#000000">1) To create algorithms for "Knowledge Tracing," the modeling of student knowledge over time.</font></div>     
<div align='left'><font size="3" color="#000000">2) To accurately predict how students will perform on future interactions.</font></div> 
<hr>
<div align='left'><font size="4" color="#000000"><h1 style="text-transform: uppercase"><strong> Purpose: </strong> </font></div></h1>
<div align='left'><font size="3" color="#000000">1) Help tackle global challenges in education.</font></div>     
<div align='left'><font size="3" color="#000000">2) Using good ML model,it is possible that any student with an Internet connection can enjoy the benefits of a personalized learning experience.</font></div> 




<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:30px;background:black; border:0; color:white' role="tab" aria-controls="home"><center>Evaluation Metrics: ROC-AUC Score</center></h2> 
<div align='left'><font size="3" color="#000000">Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.</font></div> 
<div align='left'><font size="4" color="#000000"><h1 style="text-transform: uppercase"><strong> What is AUC-ROC Curve? </strong> </font></div></h1>
<div align='left'><font size="3" color="#000000">The Receiver Operator Characteristic (ROC) curve is an evaluation metric for binary classification problems. It is a probability curve that plots the TPR against FPR at various threshold values and essentially separates the ‘signal’ from the ‘noise’. The Area Under the Curve (AUC) is the measure of the ability of a classifier to distinguish between classes and is used as a summary of the ROC curve.
<strong>ROC-AUC Score is the best measure for binary classification and imbalance class problem. </strong>This iste gives more information about AUC ROC curve<a href="https://www.analyticsvidhya.com/blog/2020/06/auc-roc-curve-machine-learning/" target="_blank">AUC-ROC-Curve.</a> </font></div> 
<img src='https://classeval.files.wordpress.com/2015/06/roc-balanced-imbalanced.png?w=768&h=302' width='700'>



<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:30px;background:black; border:0; color:white' role="tab" aria-controls="home"><center>Table Contents</center></h2> 
    
<div align='left'><font size="3" color="#000000">1. Exploratory Data Analysis</font></div>
<div align='left'><font size="3" color="#000000">2. Building Baseline Models</font></div>
<div align='left'><font size="3" color="#000000">3. Ensembling</font></div>
<div align='left'><font size="3" color="#000000">4. Conclusion</font></div>
    

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:30px;background:black; border:0; color:white' role="tab" aria-controls="home"><center>1. Exploratory Data Analysis</center></h2> 

    
    
<div align='left'><font size="4" color="#000000"><h1 style="text-transform: uppercase"><strong>Importing Necessary Libraries</strong> </font></div>    


In [ ]:
import numpy as np 
import pandas as pd 
from collections import Counter
import pandas_profiling as pp
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter('ignore')

In [ ]:
import riiideducation
env = riiideducation.make_env()

 <div align='left'><font size="4" color="#000000"><h1 style="text-transform: uppercase"><strong>Data Description</strong> </font></div>    
<hr>
<div align='left'><font size="3" color="#000000"><strong>1. row_id:</strong>ID code for the row.</font></div>    
<div align='left'><font size="3" color="#000000"><strong>2. timestamp: </strong>time between this user interaction and the first event.</font></div>    
<div align='left'><font size="3" color="#000000"><strong>3. user_id: </strong>ID code for the user</font></div>    
<div align='left'><font size="3" color="#000000"><strong>4. content_id:</strong>ID code for the user interaction</font></div>    
<div align='left'><font size="3" color="#000000"><strong>5. content_type_id: </strong>0 if question else 1 for Lecture.</font></div>    
<div align='left'><font size="3" color="#000000"><strong>6. user_answer: </strong>the user's answer to the question, Read -1 as null, for lectures. </font></div>
<div align='left'><font size="3" color="#000000"><strong>7. user_answer: </strong>IF the user responded correctly. Read -1 as null, for lectures.</font></div>
<div align='left'><font size="3" color="#000000"><strong>8. prior_question_elapsed_time: </strong>How long it took a user to answer their previous question bundle, ignoring any lectures in between. It is the total time a user took to solve all questions in the previous bundle. </font></div>
<div align='left'><font size="3" color="#000000"><strong>9. prior_question_had_explanation: </strong>Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between.</font></div>



<div align='left'><font size="4" color="#000000"><h1 style="text-transform: uppercase"><strong>Loading Dataset</strong> </font></div> 

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', low_memory=False,nrows=10**5,  dtype={'row_id': 'int64',
    'timestamp': 'int64','user_id': 'int32','content_id': 'int16', 'content_type_id': 'int8','task_container_id': 'int16',
    'user_answer': 'int8','answered_correctly': 'int8','prior_question_elapsed_time': 'float32', 'prior_question_had_explanation': 'boolean'} )    
train.head()

In [ ]:
pp_train = pp.ProfileReport(train)
pp_train

<div align='left'><font size="4" color="#000000"><h1 style="text-transform: uppercase"><strong>Data Cleaning</strong> </font></div>    
<div align='left'><font size="3" color="#000000"><strong>Removing Unwanted columns</strong> </font></div>    


In [ ]:
train = train.drop('user_answer',axis=1)
train = train.query('answered_correctly != -1').reset_index(drop=True)
train.info()

<div align='left'><font size="3" color="#000000"><strong>Missing Value Detection </strong> </font></div>    


In [ ]:
def missing(df):
    total = df.isnull().sum().sort_values(ascending = False)
    total = total[total>0]
    percent = df.isnull().sum().sort_values(ascending = False)/len(df)*100
    percent = percent[percent>0]
    return pd.concat([total, percent], axis=1, keys=['Total','Percentage'])
missing(train)

<div align='left'><font size="3" color="#000000"><strong>Missing Value Treatment</strong> </font></div>    


In [ ]:
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].fillna(train['prior_question_had_explanation'].mode()[0])
train['prior_question_elapsed_time'] = train['prior_question_elapsed_time'].fillna(train['prior_question_elapsed_time'].mean())
missing(train)

In [ ]:
train.describe()

In [ ]:
Counter(train['answered_correctly'])

In [ ]:
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype(float)
y = train['answered_correctly']
X = train.drop('answered_correctly', axis=1)

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:30px;background:black; border:0; color:white' role="tab" aria-controls="home"><center>2. Building Baseline Models</center></h2> 



## MODEL:1 Light Gradient Boost Classifier

In [ ]:
cat_col = ['user_id', 'content_type_id', 'task_container_id', 'prior_question_had_explanation']
lgb = LGBMClassifier(n_estimators = 200,class_weight="balanced", max_depth=-1,metric = 'auc',tree_learner = 'serial',learning_rate=0.08,
                     max_bin = 200,min_data_in_leaf = 80,num_leaves = 50, feature_fraction = 0.05,bagging_fraction = 0.4,random_state=42)  
fold = StratifiedKFold(n_splits = 5, shuffle =True)
pred = [];score_test =[];score_train =[];
for train_index , test_index in fold.split(X,y):
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]
    lgb= lgb.fit(X_train, y_train,eval_set=(X_test , y_test),eval_metric='auc',verbose=50,categorical_feature=cat_col,early_stopping_rounds= 50)
    y_pred_train = lgb.predict_proba(X_train)[:,1]
    y_pred_test = lgb.predict_proba(X_test)[:,1]
    score_train.append(roc_auc_score( y_train,y_pred_train))
    score_test.append(roc_auc_score( y_test,y_pred_test))
print('\n')
print('Mean training AUC:',np.mean(score_train))
print('Mean testing AUC:',np.mean(score_test))

## MODEL:2 CatBoost Classifier

In [ ]:
cat_col = ['user_id', 'content_type_id', 'task_container_id']
cat = CatBoostClassifier(bagging_temperature=0.717,border_count=198, depth=10, iterations=1500,
l2_leaf_reg=25,learning_rate=0.154, random_strength=0.003,eval_metric='AUC')
fold = StratifiedKFold(n_splits = 5, shuffle =True)
pred = [];score_test =[];score_train =[];
for train_index , test_index in fold.split(X,y):
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]
    cat= cat.fit(X_train, y_train,eval_set=(X_test , y_test),verbose=50,cat_features=cat_col,early_stopping_rounds= 50)
    y_pred_train = cat.predict_proba(X_train)[:,1]
    y_pred_test = cat.predict_proba(X_test)[:,1]
    score_train.append(roc_auc_score( y_train,y_pred_train))
    score_test.append(roc_auc_score( y_test,y_pred_test))
print('\n')
print('Mean training AUC:',np.mean(score_train))
print('Mean testing AUC:',np.mean(score_test))

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:30px;background:black; border:0; color:white' role="tab" aria-controls="home"><center>3. Ensembling: Blending of two models</center></h2> 



In [ ]:
lgb = lgb.fit(X, y,eval_set=(X, y),eval_metric='auc',verbose=50,categorical_feature=cat_col,early_stopping_rounds= 50)
cat = cat.fit(X, y,verbose=50,cat_features=cat_col,early_stopping_rounds= 50)
cat_pred = cat.predict(X)
lgb_pred = lgb.predict(X)

In [ ]:
blend = (0.6*lgb_pred) + (0.4*cat_pred)
roc_auc_score(y,blend)

## Creating Test environment

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test, sample_prediction) in iter_test:
    X_test = test.drop([ 'prior_group_answers_correct','prior_group_responses'],axis = 1)
    X_test['prior_question_had_explanation'] = X_test['prior_question_had_explanation'].fillna(X_test['prior_question_had_explanation'].mode()[0])
    X_test['prior_question_elapsed_time'] = X_test['prior_question_elapsed_time'].fillna(X_test['prior_question_elapsed_time'].mean())
    X_test['prior_question_had_explanation'] = X_test['prior_question_had_explanation'].astype(float)
    pred_lgb = lgb.predict(X_test)
    pred_cat = cat.predict(X_test)
    X_test['answered_correctly'] = (0.6*pred_lgb) + (0.4*pred_cat)
    env.predict(X_test.loc[X_test['content_type_id'] == 0, ['row_id', 'answered_correctly']])

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:30px;background:black; border:0; color:white' role="tab" aria-controls="home"><center>Conclusion </center></h2> 

<div align='left'><font size="3" color="#000000">1. EDA helpful to understand the overview of the dataset.</font></div>
<div align='left'><font size="3" color="#000000">2. LGBMClassifier perform better compare to other model.</font></div>
<div align='left'><font size="3" color="#000000">3. Blending improve the overall score.</font></div>

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:30px;background:black; border:0; color:white' role="tab" aria-controls="home"><center>Things to be taken care before submission </center></h2> 
<div align='left'><font size="3" color="#000000">This competition is different from most of other Kaggle Competitions.You will loop through a series of batches of questions. Once you make that prediction, you can move on to the next batch, you will receive test set data and make predictions with Kaggle's time-series API. So it is good if you refer these kerenel before submission</font></div> 
<hr> 
<div align='left'><font size="3"><a href="https://www.kaggle.com/sohier/competition-api-detailed-introduction" target="_blank">1. Competition API Detailed Introduction</a></div>
<div align='left'><font size="3"><a href="https://www.kaggle.com/sohier/quick-sample-submission" target="_blank">2. Quick Sample Submission</a></div>
<hr>   

 <div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='font-size:25px;background:lightgreen; border:0; color:green' role="tab" aria-controls="home"><center>Feel free to ask any question related to this topic. I'm happy to answer. If you like my work don't hesitate to upvote. HAPPY LEARNING :)</center></h2> 